In [3]:
pip install yfinance

#Import Library

In [17]:
import pandas as pd
import yfinance as yf
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
pio.templates.default = "plotly_white"

#Collect BBCA & BBRI Price Data

In [6]:
# Define the tickers for BBCA and BBRI
bbca_ticker = 'BBCA.JK'
bbri_ticker = 'BBRI.JK'

# Define the date range for the last quarter
start_date = '2023-01-01'
end_date = '2023-12-31'

# Fetch historical stock price data using yfinance
bbca_data = yf.download(bbca_ticker, start=start_date, end=end_date)
bbri_data = yf.download(bbri_ticker, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


#Calculate the Daily Returns

In [8]:
# Calculate daily returns
bbca_data['Daily_Return'] = bbca_data['Adj Close'].pct_change()
bbri_data['Daily_Return'] = bbri_data['Adj Close'].pct_change()

In [10]:
# Create a figure to visualize the daily returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=bbca_data.index, y=bbca_data['Daily_Return'],
                         mode='lines', name='BBCA', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=bbri_data.index, y=bbri_data['Daily_Return'],
                         mode='lines', name='BBRI', line=dict(color='green')))

fig.update_layout(title='Daily Returns for BBCA and BBRI',
                  xaxis_title='Date', yaxis_title='Daily Return',
                  legend=dict(x=0.02, y=0.95))

fig.show()

#Cumulative Returns Analysis

In [11]:
# Calculate cumulative returns for the last quarter
bbca_cumulative_return = (1 + bbca_data['Daily_Return']).cumprod() - 1
bbri_cumulative_return = (1 + bbri_data['Daily_Return']).cumprod() - 1

# Create a figure to visualize the cumulative returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=bbca_cumulative_return.index, y=bbca_cumulative_return,
                         mode='lines', name='BBCA', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=bbri_cumulative_return.index, y=bbri_cumulative_return,
                         mode='lines', name='Google', line=dict(color='green')))

fig.update_layout(title='Cumulative Returns for BBCA and BBRI',
                  xaxis_title='Date', yaxis_title='Cumulative Return',
                  legend=dict(x=0.02, y=0.95))


#Volatility Analysis

In [14]:
# Calculate historical volatility (standard deviation of daily returns)
bbca_volatility = bbca_data['Daily_Return'].std()
bbri_volatility = bbri_data['Daily_Return'].std()

# Create a figure to compare volatility
fig1 = go.Figure()
fig1.add_bar(x=['BBCA', 'BBRI'], y=[bbca_volatility, bbri_volatility],
             text=[f'{bbca_volatility:.4f}', f'{bbri_volatility:.4f}'],
             textposition='auto', marker=dict(color=['blue', 'green']))

fig1.update_layout(title='Volatility Comparison',
                   xaxis_title='Stock', yaxis_title='Volatility (Standard Deviation)',
                   bargap=0.5)


#Comparing with IHSG (Finding

In [15]:
ihsg_data = yf.download('^JKSE', start=start_date, end=end_date)  # IHSG index as the market benchmark

# Calculate daily returns for both stocks and the market
bbca_data['Daily_Return'] = bbca_data['Adj Close'].pct_change()
bbri_data['Daily_Return'] = bbri_data['Adj Close'].pct_change()
ihsg_data['Daily_Return'] = ihsg_data['Adj Close'].pct_change()

# Calculate Beta for BBCA and BBRI
cov_bbca = bbca_data['Daily_Return'].cov(ihsg_data['Daily_Return'])
var_market = ihsg_data['Daily_Return'].var()

beta_bbca = cov_bbca / var_market

cov_bbri = bbri_data['Daily_Return'].cov(ihsg_data['Daily_Return'])
beta_bbri = cov_bbri / var_market

# Compare Beta values
if beta_bbca > beta_bbri:
    conclusion = "BBCA is more volatile (higher Beta) compared to BBRI."
else:
    conclusion = "BBRI is more volatile (higher Beta) compared to BBCA."

# Print the conclusion
print("Beta for BBCA:", beta_bbca)
print("Beta for BBRI:", beta_bbri)
print(conclusion)

[*********************100%%**********************]  1 of 1 completed
Beta for BBCA: 0.9086019901666547
Beta for BBRI: 1.2441439421062075
BBRI is more volatile (higher Beta) compared to BBCA.


In [18]:
# Create a bar chart for Beta values using Plotly Express
data = {'Stock': ['BBCA', 'BBRI'], 'Beta': [beta_bbca, beta_bbri]}
fig = px.bar(data, x='Stock', y='Beta', color='Stock', text='Beta',
             labels={'Beta': 'Beta Value'},
             title='Beta Comparison: BBCA vs. BBRI')

# Show the plot
fig.show()